In [13]:
%matplotlib inline
import itertools
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras import losses
from keras.datasets import mnist
from keras.models import Sequential, model_from_json

from keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, MaxPooling2D, LeakyReLU
from keras import backend as K
from keras.optimizers import Adam

#from keras.utils.training_utils import multi_gpu_model

from helpers import *

In [3]:
print(keras.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2.0.9
[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17942444580097936080
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 355729408
locality {
  bus_id: 1
}
incarnation: 200314842381896196
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:17.0"
, name: "/gpu:1"
device_type: "GPU"
memory_limit: 355729408
locality {
  bus_id: 1
}
incarnation: 3773727828731387387
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:00:18.0"
, name: "/gpu:2"
device_type: "GPU"
memory_limit: 355729408
locality {
  bus_id: 1
}
incarnation: 4396736140880381404
physical_device_desc: "device: 2, name: Tesla K80, pci bus id: 0000:00:19.0"
, name: "/gpu:3"
device_type: "GPU"
memory_limit: 357695488
locality {
  bus_id: 1
}
incarnation: 16603126585749387079
physical_device_desc: "device: 3, name: Tesla K80, pci bus id: 0000:00:1a.0"
, name: "/gpu:4"
device_type: "GPU"
memory_limit: 357695488
locality {
  bus_id: 1
}
incarnation: 1807204115

In [14]:
imgs, gt_imgs = training_dataset(limit=100)
print('Image size =', imgs[0].shape)

Loading 100 training images
Loading 100 groundtruth images
Image size = (400, 400, 3)


In [16]:
patch_size = 16 # each patch is 16*16 pixels

In [11]:
train_data = [
    (patch, gt_patch)
    for i in range(len(imgs))
    for patch, gt_patch initertools.islice(gen_random_patches(reflect_border(imgs[i], patch_size, 2),
                                                              reflect_border(gt_imgs[i], patch_size, 2), i), 25*25*8)]


X = np.asarray([img for img, _ in train_data])
y = np.asarray([value_to_class(gt_patch) for _, gt_patch in train_data]).reshape((-1, 1))

print(X.shape)
print(y.shape)

(500000, 80, 80, 3)
(500000, 1)


In [19]:
# Model parameters

batch_size = 64 * 8
num_filters_1 = 16
num_filters_2 = 32
num_filters_3 = 64
num_filters_4 = 128

epochs = 50

In [31]:
def build_model(gpus=0):
    model = Sequential()
    model.add(Conv2D(num_filters_1, kernel_size=(4, 4), input_shape=(80, 80, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(num_filters_2, kernel_size=(4,4)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(num_filters_3, kernel_size=(4,4)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(num_filters_4, kernel_size=(4,4)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    simple_model = model
    if gpus > 1:
        model = multi_gpu_model(simple_model, gpus=gpus)

    model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])
    
    return (model, simple_model)

In [32]:
model, simple_model = build_model(0)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 77, 77, 16)        784       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 77, 77, 16)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 38, 38, 16)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 38, 38, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 35, 35, 32)        8224      
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 35, 35, 32)        0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 17, 17, 32)        0         
__________

In [88]:
model.fit(X, y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True)

Epoch 1/50
500000/500000 [==============================] - 78s 157us/step - loss: 0.3397 - acc: 0.8431
Epoch 2/50
500000/500000 [==============================] - 79s 159us/step - loss: 0.1942 - acc: 0.9201
Epoch 3/50
500000/500000 [==============================] - 78s 155us/step - loss: 0.1604 - acc: 0.9351
Epoch 4/50
500000/500000 [==============================] - 78s 156us/step - loss: 0.1404 - acc: 0.9437
Epoch 5/50
500000/500000 [==============================] - 78s 155us/step - loss: 0.1271 - acc: 0.9498
Epoch 6/50
500000/500000 [==============================] - 78s 155us/step - loss: 0.1191 - acc: 0.9531
Epoch 7/50
500000/500000 [==============================] - 77s 155us/step - loss: 0.1121 - acc: 0.9559
Epoch 8/50
500000/500000 [==============================] - 77s 154us/step - loss: 0.1059 - acc: 0.9583
Epoch 9/50
500000/500000 [==============================] - 77s 155us/step - loss: 0.1022 - acc: 0.9598
Epoch 10/50
500000/500000 [==============================] - 78s

In [30]:
# save the model and the weights to files
model_json = simple_model.to_json()
with open('savedModels/model_18', 'w') as f:
    f.write(model_json)

simple_model.save_weights('savedModels/weights_18')